# Data Acquisition and Cleaning

# Ideas

Cricket is a game of numbers. So much can be mined from this rich datasets.

1. Trivia
2. Fantasy teams
3. Player Rankings
4. Simulate games between 2 teams: Imput 2 teams and generate scorecards from distribution

In [20]:
# import necessary libraries
import pandas as pd
import numpy as np
import requests
import wikipedia
import re
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen
from scrapy import selector
import datetime as dt
import pickle
from skimage import io
from IPython.display import clear_output
import pdb
plt.style.use('ggplot')
%matplotlib inline

# Test Matches

In [13]:
# Collect Test Metadata
ALL_TEST = {}
missed_year = []
for year in range(1901,2021):
    
    if year%10 == 0:
        clear_output()
    print(year)
    base_scorecard_link = 'http://stats.espncricinfo.com/'
    my_url = f'http://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=1;id={year};type=year'
    r = requests.get(my_url)
    soup = BeautifulSoup(r.content,'lxml')

    HEADER = soup.find('thead')
    cols = []
    try:
        for header in HEADER.find_all('th'):
            cols.append(header.text.lower())

        cols.append('scorecard_link')
        test_matches = pd.DataFrame(columns = cols)
        

        # Now get information from Table
        table = soup.find_all('tbody')
        len(table)
        games = table[0].find_all('tr')
        for i,game in enumerate(games):
            for j,g in enumerate(game.find_all('td')):
                col = cols[j]
                test_matches.loc[i,col] = g.text.lower()
            test_matches.loc[i,'scorecard_link'] = base_scorecard_link + g.a['href']
            
        test_matches.rename(columns={"scorecard": "test_index"}, inplace=True)
        test_matches.set_index('test_index', inplace=True)
        ALL_TEST[year] = test_matches
    except AttributeError:
        print(f'Information for {year} missing!')
        missed_year.append(year)
        ALL_TEST[year] = pd.DataFrame()
    


2020


In [14]:
ALL_TEST[1999]

,team 1,team 2,winner,margin,ground,match date,scorecard_link
test_index,,,,,,,
test # 1438,new zealand,india,drawn,,hamilton,"jan 2-6, 1999",http://stats.espncricinfo.com//ci/engine/match...
test # 1439,australia,england,australia,98 runs,sydney,"jan 2-5, 1999",http://stats.espncricinfo.com//ci/engine/match...
test # 1440,south africa,west indies,south africa,149 runs,cape town,"jan 2-6, 1999",http://stats.espncricinfo.com//ci/engine/match...
test # 1441,south africa,west indies,south africa,351 runs,centurion,"jan 15-18, 1999",http://stats.espncricinfo.com//ci/engine/match...
test # 1442,india,pakistan,pakistan,12 runs,chennai,"jan 28-31, 1999",http://stats.espncricinfo.com//ci/engine/match...
test # 1443,india,pakistan,india,212 runs,delhi,"feb 4-7, 1999",http://stats.espncricinfo.com//ci/engine/match...
test # 1444,india,pakistan,pakistan,46 runs,kolkata,"feb 16-20, 1999",http://stats.espncricinfo.com//ci/engine/match...
test # 1445,sri lanka,india,drawn,,colombo (ssc),"feb 24-28, 1999",http://stats.espncricinfo.com//ci/engine/match...
test # 1446,new zealand,south africa,drawn,,auckland,"feb 27-mar 3, 1999",http://stats.espncricinfo.com//ci/engine/match...


In [15]:
sum = 0
for key in ALL_TEST.keys():
    sum+=len(ALL_TEST[key])
print(sum*4.2/365, "years of cricket")

27.087123287671236 years of cricket


# ODI

In [18]:
# Collect Test Metadata
ALL_ODI = {}
missed_year = []
for year in range(1971,2021):
    
    if year%10 == 0:
        clear_output()
    print(year)
    base_scorecard_link = 'http://stats.espncricinfo.com/'
    my_url = f'http://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id={year};type=year'
    r = requests.get(my_url)
    soup = BeautifulSoup(r.content,'lxml')

    HEADER = soup.find('thead')
    cols = []
    try:
        for header in HEADER.find_all('th'):
            cols.append(header.text.lower())

        cols.append('scorecard_link')
        matches = pd.DataFrame(columns = cols)

        # Now get information from Table
        table = soup.find_all('tbody')
        len(table)
        games = table[0].find_all('tr')
        for i,game in enumerate(games):
            for j,g in enumerate(game.find_all('td')):
                col = cols[j]
                matches.loc[i,col] = g.text.lower()
            matches.loc[i,'scorecard_link'] = base_scorecard_link + g.a['href']
        
        matches.rename(columns={"scorecard": "odi_index"}, inplace=True)
        matches.set_index('odi_index', inplace=True)

        ALL_ODI[year] = matches
    except AttributeError:
        print(f'Information for {year} missing!')
        missed_year.append(year)
        ALL_ODI[year] = pd.DataFrame()
    


2020


In [19]:
ALL_ODI[1999]

,team 1,team 2,winner,margin,ground,match date,scorecard_link
odi_index,,,,,,,
odi # 1378,new zealand,india,new zealand,5 wickets,taupo,"jan 9, 1999",http://stats.espncricinfo.com//ci/engine/match...
odi # 1379,australia,england,england,7 runs,brisbane,"jan 10, 1999",http://stats.espncricinfo.com//ci/engine/match...
odi # 1380,england,sri lanka,england,4 wickets,brisbane,"jan 11, 1999",http://stats.espncricinfo.com//ci/engine/match...
odi # 1381,new zealand,india,india,2 wickets,napier,"jan 12, 1999",http://stats.espncricinfo.com//ci/engine/match...
odi # 1382,australia,sri lanka,australia,8 wickets,sydney,"jan 13, 1999",http://stats.espncricinfo.com//ci/engine/match...
...,...,...,...,...,...,...,...
odi # 1527,zimbabwe,sri lanka,no result,,bulawayo,"dec 11, 1999",http://stats.espncricinfo.com//ci/engine/match...
odi # 1528,zimbabwe,sri lanka,sri lanka,13 runs,bulawayo,"dec 12, 1999",http://stats.espncricinfo.com//ci/engine/match...
odi # 1529,zimbabwe,sri lanka,sri lanka,98 runs,harare,"dec 15, 1999",http://stats.espncricinfo.com//ci/engine/match...


In [ ]:
pickle.dump(ALL_TEST,open('ALL_TEST', "wb" ))
pickle.dump(ALL_TEST,open('ALL_OD', "wb" ))



In [102]:
sum = 0
for key in ALL_ODI.keys():
    sum+=len(ALL_ODI[key])
print(sum, "days of cricket")

4233 days of cricket


# Get Match Data

In [33]:
outs = ['b', 'c', 'lbw', 'st', 'hw', 'ro', 'hand', 'obs']

In [112]:
def get_math_data(link, _format):
    
    if _format.lower() == 'test':
        pass
    if _format.lower() == 'odi':
        pass
    
def how_out(dismissal):
    """Returns triplet tuple as (how, fielder, bowler)
    """
    
    dismissal = dismissal.lower()
    
    bowler = re.findall(r'b\s(\w+\s{1}\w+)\s?', dismissal)
    fielder = re.findall(r'c\s([\w\s]+)\sb', dismissal)
    
    if len(fielder) == 0:
        print('stumped!')
        fielder = re.findall(r'st\s([\w\s]+)\sb', dismissal)
    
    if dismissal.find('(sub')>-1:
        fielder = ['substitute']
        
    
    print(bowler, fielder)
    
    if len(bowler) > 1:
        bowler = bowler[0]
    if len(fielder) > 1:
        fielder = fielder[0]
    
    # handling the ball
    if dismissal.find('hand') > -1:
        fielder = ''
        blower = ''
        return 'hand', fielder, bowler

    # obstructing the fieldsman
    if dismissal.find('obstruct') > -1:
        fielder = ''
        blower = ''
        return 'obs', fielder, bowler
    
    # run out
    if dismissal.find('run out') > -1:
        fielder = ''
        blower = ''
        return 'ro', fielder, bowler
    
    # caught and bowled
    if dismissal.find('&') > -1:
        fielder = bowler
        return 'c', fielder, bowler
    
    # LBW
    if dismissal.find('lbw') > -1 or dismissal.find('leg before')>-1:
        fielder = ''
        return 'lbw',fielder, bowler
    
    # Hit wicket
    if dismissal.find('hit wicket') > -1:
        fielder = ''
        return 'hw',fielder, bowler
    
    # Stumped
    if dismissal.find('st') > -1:
        return 'st',fielder, bowler
    
#     if bowler == None:
#         print('Warning! Error')
#         return '','',''
    
    # caught
    if len(fielder)>-1 and len(bowler)> -1:
        return 'c', fielder, bowler
    
    # else bowled!
    fielder = ''
    return 'b',fielder, bowler

In [114]:
dismissal = 'lbw b Curtley Ambrose'
print(how_out(dismissal))

stumped!
['curtley ambrose'] []
('lbw', '', ['curtley ambrose'])


In [109]:
dismissal.find('(sub')

-1

In [126]:
# soup.find('',class_="")
link = ALL_ODI[1992].iloc[45,-1]
print(link)
r = requests.get(link)
soup = BeautifulSoup(r.content,'lxml')

match_metadata = soup.find('div', class_="sub-module gameHeader gameHeader--fullWidth")
score_info = soup.find('div' ,class_ = "col-b")

first_innings_data = soup.find_all('article',class_="sub-module scorecard")[0]
second_innings_data = soup.find_all('article',class_="sub-module scorecard")[1]
match_data = soup.find_all('article', class_="sub-module game-information pre")




http://stats.espncricinfo.com//ci/engine/match/65148.html


In [128]:
first_innings_data.text

'England InningsBATSMENRBM4s6sSRAJ Stewart (c) †c Harris b Patel4159777069.49IT Botham b Patel825211032.00GA Hickc Greatbatch b Harris5670946180.00RA Smithc Patel b Jones3872673052.77AJ Lambc Cairns b Watson1229390041.37CC Lewisc & b Watson011000.00DA Reevenot out2127371077.77DR Pringlec sub (RT Latham) b Jones1016200062.50PAJ DeFreitasc Cairns b Harris013000.00RK Illingworthnot out22300100.00Extras12 (b 1, lb 7, w 4)TOTAL200/8 (50 Overs, RR: 4.0)Did not bat: GC SmallFall of wickets: 1-25 (Sir Ian Botham), 2-95 (Alec Stewart), 3-135 (Graeme Hick), 4-162 (Robin Smith), 5-162 (Chris Lewis), 6-169 (Allan Lamb), 7-189 (Derek Pringle), 8-195 (Phil DeFreitas)BowlingOMRWEconWDNBDN Patel1012622.6000CZ Harris803924.8710W Watson1004024.0010CL Cairns302107.0010GR Larsen1032402.4010AH Jones904224.6600'